# Thunor Tutorial

Thunor is a Python package for managing and viewing multi-timepoint high throughput screen data. It utilizes the drug-induced proliferation rate (DIP rate) metric, which is a dynamic measure of drug response.

The Python package also has a web interface available, called Thunor Web.

## [Ignore] Set up code for documentation

In [ ]:
# Temporary hack, see https://github.com/poliastro/poliastro/issues/281
from IPython.display import HTML
HTML('<script type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.1.10/require.min.js"></script>')

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath('../'))

## Start Jupyter Notebook

Run jupyter notebook with the following argument:

    jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10
    
The data rate limit needs to be increased or `init_notebook_mode()` throws an error.

## Load a file

First, specify a file to load. This can be done as a string filename, but here we load an example dataset from the thunor package.

In [ ]:
import pkg_resources
vu001_file = pkg_resources.resource_filename('thunor', 'testdata/VU001.h5')

Load the file using `read_hdf` (for HDF5 files), `read_vanderbilt_hts` (for CSV files), or another appropriate reader.

In [ ]:
from thunor.io import read_hdf
vu001 = read_hdf(vu001_file)

We'll just use a subset of the drugs, to make the plots manageable.

In [ ]:
vu001 = vu001.filter(drugs=['cediranib', 'everolimus', 'paclitaxel'])

In [ ]:
vu001.drugs

In [ ]:
vu001.cell_lines

## Calculate DIP rates and parameters

These two operations can be done in two lines of code (plus imports).

In [ ]:
from thunor.dip import dip_rates, dip_fit_params

ctrl_dip_data, expt_dip_data = dip_rates(vu001)
fit_params = dip_fit_params(ctrl_dip_data, expt_dip_data)

## Setting up plots

Each of the `plot_X` functions returns a plotly `Figure` object which can be visualised in a number of ways. Here, we use the offline `iplot` function, which generates a plot for use with Jupyter notebook. We could also generate plots using the `plot` function in standalone HTML files. See the [plotly documentation](https://plot.ly/python/offline/) for more information on the latter approach. 

In [ ]:
from thunor.plots import plot_dip, plot_dip_params, plot_time_course, plot_ctrl_dip_by_plate
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode()

## Plot Types

### Plot DIP rate curves

In [ ]:
iplot(plot_dip(fit_params))

### Plot DIP parameters

In [ ]:
iplot(plot_dip_params(fit_params, 'auc'))

### Filtering fit_params

The `fit_params` object is a pandas data frame, so we can filter it before plotting. Some examples:

In [ ]:
fit_params_bt20 = fit_params.iloc[fit_params.index.isin(['BT20'], level='cell_line'), :]
fit_params_abemaciclib = fit_params.iloc[fit_params.index.isin(['abemaciclib'], level='drug'), :]

iplot(plot_dip(fit_params_bt20))

### Plot time course

Time course plot for abemaciclib on BT20 cells:

In [ ]:
iplot(plot_time_course(
    vu001.filter(drugs=['paclitaxel'], cell_lines=['BT20'])
))

### Quality control check: plot DIP rate ranges by cell line and plate (box plot)

In [ ]:
iplot(plot_ctrl_dip_by_plate(ctrl_dip_data))